In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive')
!pwd
os.chdir('gdrive/My Drive/MGNets/')
!pwd
!ls

### Prepare for Input


In [ ]:
import sklearn, sklearn.datasets
import sklearn.naive_bayes, sklearn.linear_model, sklearn.svm, sklearn.neighbors, sklearn.ensemble
import matplotlib.pyplot as plt
import scipy.sparse
import numpy as np
import time, re
import pickle as pkl
import pandas as pd
import scipy.io as sio
from  scipy import *
import random

from sklearn.model_selection import StratifiedKFold


def data_projection(data, U):
  U_trans = np.transpose(U)
  M_U,_ = U_trans.shape
  N,V,M,F = data.shape
  new_data = np.zeros([N,V,M_U,F])
  for i in range(N):
    for v in range(V):
      data_iv = data[i,v,:,:]
      data_iv_trans = np.matmul(U_trans,data_iv)
      # data_iv_trans = np.matmul(data_iv_trans, U)
      new_data[i,v,:,:] = data_iv_trans

  return new_data    



def convert_minus_one_2_zero(label):
  for i in range(len(label)):
    if(label[i]<0):
      label[i] = 0;
  return label    

def load_data(f):
  k = 0
  input_data = {}

  for line in f:
    input_data[k] = np.int32(line.split())
    k = k + 1
  
  return input_data


def load_train_index(f):
  train_index = []
  k = 0
  for line in f:
    train_index.append(np.int32(line))

  return train_index
    


def load_data_my_new(data_name, train_fold_chosen):
  # load data from google drive
  if(data_name == 'BP'):
    x = scipy.io.loadmat('BP.mat')
    X_normalize = x['X_normalize']
    label = x['label']
  elif(data_name == 'HIV'):
    x = scipy.io.loadmat('HIV.mat') 
    X_normalize = x['X'] 
    label = x['label']
  elif(data_name == 'PPMI'):
    x = scipy.io.loadmat('PPMI.mat') 
    X_normalize = x['X'] 
    label = x['label']

  # reshape data_size to N,V,M,F  
  N_subject = X_normalize.size
  X_1 = X_normalize[0][0]
  M,F,V = X_1.shape
  print('loading data:',data_name,'of size:',N_subject,V,M,F)
  data = np.zeros([N_subject,V,M,F])
  for i in range(N_subject):
    X_i = X_normalize[i][0]
    for j in range(V):
      X_ij = X_i[:,:,j]
      data[i,j,:,:] = X_ij

  labels = np.array(label)
  if(data_name == 'BP'):
    f_test = open('results/BP_divide_test_index.txt','r')
    test_idx = load_data(f_test)
    f_train = open('results/BP_divide_train_index.txt','r')
    train_idx = load_data(f_train)
  elif(data_name == 'HIV'):
    f_test = open('results/HIV_divide_test_index.txt','r')
    test_idx = load_data(f_test)
    f_train = open('results/HIV_divide_train_index.txt','r')
    train_idx = load_data(f_train)
  elif(data_name == 'PPMI'):
    f_test = open('results/PPMI_divide_test_index.txt','r')
    test_idx = load_data(f_test)
    f_train = open('results/PPMI_divide_train_index.txt','r')
    train_idx = load_data(f_train)  

  f_test.close()
  f_train.close()

  train_set_ratio = 0.8
  train_chosen_idx = train_idx[train_fold_chosen]
  data_size = train_chosen_idx.size;  
  train_data_size = np.floor(data_size*train_set_ratio)
  train_index = random.sample(list(train_chosen_idx), int(train_data_size))
  test_index = np.setdiff1d(train_chosen_idx,train_index)

  labels_set = list()
  indexes_set = list()

  train_label, test_label = labels[train_index], labels[test_index]
  val_label,val_index = test_label,test_index
  train_label = convert_minus_one_2_zero(train_label)
  val_label = convert_minus_one_2_zero(val_label)
  test_label = convert_minus_one_2_zero(test_label)

  train_label = np.array(train_label).flatten()
  val_label = np.array(val_label).flatten()
  test_label = np.array(test_label).flatten()

  train_index = np.array(train_index).flatten()
  val_index = np.array(val_index).flatten()
  test_index = np.array(test_index).flatten()

  labels_set.append((train_label,val_label,test_label))
  indexes_set.append((train_index,val_index,test_index))

  train_index_all = train_idx
  test_index_all = test_idx

  subj = 0
  return data, subj, indexes_set, labels_set, train_index_all, test_index_all, convert_minus_one_2_zero(labels)


### Re-implement graph.py to construct ajacency matrix A and normalized laplacian matrix L_hat (Graph.py)

In [ ]:
### re-implement graph.py to construct ajacency matrix A and normalized laplacian matrix L_hat

import sklearn.metrics
import sklearn.neighbors
import scipy
import numpy as np
from scipy.sparse import rand
import scipy.sparse as sp
from scipy.sparse import isspmatrix
from scipy.sparse.linalg.eigen.arpack import eigsh




### Kif's code
def normalize_adj(adj):
    """Symmetrically normalize adjacency matrix."""
    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    return adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()


def preprocess_adj(adj):
    """Preprocessing of adjacency matrix for simple GCN model and conversion to tuple representation."""
    adj_normalized = normalize_adj(adj + sp.eye(adj.shape[0]))
    return adj_normalized


### construct ajacency matrix
def compute_dist(graph,k,metric):
  dist = scipy.spatial.distance.pdist(graph,metric)
  dist_square = scipy.spatial.distance.squareform(dist)
  id_dist = np.argsort(dist_square)[:,:k+1]
  dist_square.sort()
  dist_square = dist_square[:,:k+1]
  return dist_square,id_dist


def build_ajacency(dist,idx):
  M,k = dist.shape
  sigma = np.mean(dist[:,-1])**2
  dist = np.exp(-dist/sigma)

  # construct sparse matrix
  I = np.arange(0,M).repeat(k)
  J = idx.reshape(M*k)
  V = dist.reshape(M*k)
  W = scipy.sparse.coo_matrix((V,(I,J)), shape = (M,M))
  W.setdiag(0)
  # construct symmetric matrix
  bigger_index = W.T>W
  # W = W - W.multiply(bigger_index) + W.T.multiply(bigger_index)
  W = W + W.T.multiply(bigger_index)
  assert(type(W)) is scipy.sparse.csr.csr_matrix
  
  return W

def build_laplacian(W, normalized = True):
  d = W.sum(axis = 0)
  if not normalized:
    D = scipy.sparse.diags(d.A.squeeze(), 0)
    L = D - W
  else:   
    d += np.spacing(np.array(0,W.dtype))
    d = 1/np.sqrt(d)
    D = scipy.sparse.diags(d.A.squeeze(),0)
    I = scipy.sparse.identity(d.size,dtype= W.dtype)
    L = I - D*W*D
    # np.savetxt(r'results/PPMI/PPMI_Laplacian.txt',L.todense(),fmt="%f")
    # print('I am writing！！！！')
    #L = scipy.sparse.rand(d.size, d.size, density=0.25, format="csr", random_state=42)

  # assert(type(L)) is scipy.sparse.csr_matrix
  return L

def rescale_L(L, l_max):
  L = 2/l_max * L
  I = scipy.sparse.identity(L.shape[0], format='csr', dtype=L.dtype)
  L_hat = L - I
  return L_hat


def obtain_normalized_adj(node_features, k, metric = 'euclidean'):
    dist, idx = compute_dist(node_features, k, metric = 'euclidean')
    A = build_ajacency(dist, idx).astype(np.float32)
    I = scipy.sparse.eye(A.shape[0],dtype= A.dtype)
    A_hat = A + I

    d = A_hat.sum(axis = 0)
    d += np.spacing(np.array(0,A_hat.dtype))
    d = 1/np.sqrt(d)
    D = scipy.sparse.diags(d.A.squeeze(),0)
    normalized_A = D*A_hat*D

    return normalized_A


def chebyshev_polynomials(adj, k):
    """
    Calculate Chebyshev polynomials up to order k. Return a list of sparse matrices (tuple representation).
    """
    print("Calculating Chebyshev polynomials up to order {}...".format(k))

    adj_normalized = normalize_adj(adj)
    laplacian = sp.eye(adj.shape[0]) - adj_normalized
    largest_eigval, _ = eigsh(laplacian, 1, which='LM')
    scaled_laplacian = (2. / largest_eigval[0]) * laplacian - sp.eye(adj.shape[0])

    t_k = list()
    t_k.append(sp.eye(adj.shape[0]))
    t_k.append(scaled_laplacian)

    def chebyshev_recurrence(t_k_minus_one, t_k_minus_two, scaled_lap):
        s_lap = sp.csr_matrix(scaled_lap, copy=True)
        return 2 * s_lap.dot(t_k_minus_one) - t_k_minus_two

    for i in range(2, k+1):
        t_k.append(chebyshev_recurrence(t_k[-1], t_k[-2], scaled_laplacian))

    return sparse_to_tuple(t_k)
    
# node_features = np.random.rand(4,4)
# k = 2
# normalized_A = obtain_normalized_adj(node_features,k)## Either code works fine to obtain the normalzed A
# normalized_A2 = preprocess_adj(A)

### Initialization Weights and Bias

In [ ]:
import tensorflow as tf

initializer = tf.initializers.glorot_uniform()

def zeros(shape, name=None):
    initial = tf.zeros(shape, dtype=tf.float32)
    return tf.Variable(initial, name=name, trainable = True, dtype=tf.float32)

def get_weight( shape , name ='None' ):
    # return tf.Variable(  tf.random.truncated_normal(shape,stddev=0.1) , name= name , trainable = True , dtype=tf.float32 )
    return tf.Variable(  initializer(shape) , name = name, trainable=True , dtype=tf.float32 )

def initialize_bias_weights(shapes):
  weights = []
  for i in range( len( shapes ) ):
    weights.append( get_weight( shapes[ i ] , 'weight{}'.format( i )) )

  bias = []  
  for i in range(len(shapes)):
    bias_shape = shapes[i][1] 
    bias_i = zeros(bias_shape, 'bias{}'.format( i ))
    bias.append(bias_i)

  return weights, bias  

def initialize_shape(num_layers, Input_dim, Output_dim, M, process):
  shapes = []

  for i in range(num_layers):
    if(i == 0):
      shapes_i = [Input_dim, Output_dim[0]]
    elif(i == num_layers - 1):
      if(process == 'concat'):
        shapes_i = [M * Output_dim[i-1], Output_dim[i]]
      else:
        shapes_i = [Output_dim[i-1], Output_dim[i]]

    else:
      shapes_i = [Output_dim[i-1], Output_dim[i]]  

    shapes.append(shapes_i)

  return shapes    

# weights, bias = initialize_bias_weights(shapes)

# num_layers = 2
# Input_dim = 80
# M = 30
# process = 'concat'
# Output_dim = [60,2]
# shapes = initialize_shape(num_layers, Input_dim, Output_dim, M, process)
# print(shapes)

### Generate Noise

In [ ]:
import numpy as np

def reshape_data(data):
  N,M,F,V = data.shape
  data_reshape = np.zeros([N,V,M,F])
  for i in range(N):
    X_i = data[i,:,:,:]
    for j in range(V):
      X_ij = X_i[:,:,j]
      data_reshape[i,j,:,:] = X_ij

  return data_reshape    

def reshape_data_back(data):
  N,V,M,F = data.shape
  data_reshape = np.zeros([N,M,F,V])
  for i in range(N):
    X_i = data[i,:,:,:]
    for v in range(V):
      X_ij = X_i[v,:,:]
      data_reshape[i,:,:,v] = X_ij

  return data_reshape  


def load_data_only(data_name):
  # load data from google drive
  if(data_name == 'BP'):
    x = scipy.io.loadmat('BP.mat')
    X_normalize = x['X_normalize']
  elif(data_name == 'HIV'):
    x = scipy.io.loadmat('HIV.mat') 
    X_normalize = x['X'] 
  elif(data_name == 'PPMI'):
    x = scipy.io.loadmat('PPMI.mat') 
    X_normalize = x['X'] 

  N = X_normalize.size
  X_1 = X_normalize[0][0]
  M,F,V = X_1.shape
  data = np.zeros([N,M,F,V])
  for i in range(N):
    X_i = X_normalize[i][0]
    for j in range(V):
      X_ij = X_i[:,:,j]
      data[i,:,:,j] = X_ij

  return data  

def add_noise(data, view, sigma):
  [N,M,F,V] = data.shape;
  X = data.copy()
  if(view<V):
    X_v = data[:,:,:,view]
    s = np.random.normal(0, sigma, [N,M,F])
    X_corrupted = X_v + np.array(s)
    # print(X_corrupted - X_v)
    X[:,:,:,view] = X_corrupted
  
  elif(view == V):
    print('adding noise to both views')
    X_v1 = data[:,:,:,0]
    X_v2 = data[:,:,:,1]

    s1 = np.random.normal(0, sigma, [N,M,F])
    X_corrupted1 = X_v1 + np.array(s1)

    s2 = np.random.normal(0, sigma, [N,M,F])
    X_corrupted2 = X_v2 + np.array(s2)

    X[:,:,:,0] = X_corrupted1
    X[:,:,:,1] = X_corrupted2

  return X

# X = np.random.rand(3,3,2,2)
# X_noise = add_noise(X, 0, 0.1)
# print(X - X_noise)

### GCN.py

In [ ]:
import tensorflow as tf

def feature_process(X, process = 'concat'):
  if(process == 'concat'):
    H,W = X.shape
    X_v = tf.reshape(X, [H*W,])

  if(process == 'avg'):
    X_v = tf.reduce_mean(X,axis = 0)

  if(process == 'max'):
    X_v = tf.reduce_max(X,axis = 0)  
    
  return X_v  
  
       
def Graphconvolution_all(A, x, dropout, process):
  if(isspmatrix(A)):
    A = A.todense()

  num_layers = len(weights)-2
  if(len(weights) != len(bias)):
    print('length error of initialization')
 
  tf.random.set_seed(0)
  input = tf.nn.dropout(x, dropout)
  # input = x

  for i in range(num_layers):

    output = tf.matmul(input, weights[i])
    output = tf.matmul(A, output)
    output = output + bias[i]

    output = tf.nn.relu(output)
    input = output
  
  output = feature_process(output, process)    

  return output


def view_pooling_func(X, view_pooling):

  if(view_pooling == 'avg'):
    X_pooling = tf.reduce_mean(X,axis = 0)

  if(view_pooling == 'max'): 
    X_pooling = tf.reduce_max(X,axis = 0)

  if(view_pooling == 'concat'): 
    [V,F] = X.shape
    X_pooling = tf.reshape(X,[1,V*F])


  return X_pooling  


def MV_model_infer(inputs_all, A, dropout, process, view_pooling):
  N, V, M, F = inputs_all.shape
  outputs_all = []
  for i in range(N):
    outputs_all_v = []
    for v in range(V):
      input_i_v = inputs_all[i,v,:,:]
      output_i = Graphconvolution_all(A,input_i_v, dropout, process)
      outputs_all_v.append(output_i)

    output_feature_all_v = tf.convert_to_tensor(outputs_all_v)
    View_feature_v = view_pooling_func(output_feature_all_v, view_pooling)
    # View_feature_v = 0.018*output_feature_all_v[0,:] + 0.999 * output_feature_all_v[1,:] ## For HIV
    #View_feature_v = 0.0089*output_feature_all_v[0,:] + 0.999 * output_feature_all_v[1,:] ## For BP

    outputs_all.append(View_feature_v)

  outputs_all = tf.convert_to_tensor(outputs_all)
  
  # if(process == 'concat'):
  #   _,F_out = outputs_all.shape
  #   outputs_all = tf.reshape(outputs_all, [int(N * M), int(F_out/M)])
  #   outputs_all = tf.nn.l2_normalize(outputs_all, axis=1, epsilon=1e-12, name=None)
  #   outputs_all = tf.reshape(outputs_all, [int(N), int(F_out)])
  outputs_all = tf.nn.dropout(outputs_all, dropout)

  outputs_all = tf.matmul(outputs_all, weights[-2]) + bias[-2]
  outputs_all = tf.nn.relu(outputs_all)

  logits  = tf.matmul(outputs_all, weights[-1]) + bias[-1]

  return logits  


def MV_model_infer_concat(inputs_all, A, dropout, process, view_pooling):
  N, V, M, F = inputs_all.shape
  for i in range(N):
    for v in range(V):
      input_i_v = inputs_all[i,v,:,:]
      output_i = Graphconvolution_all(A,input_i_v, dropout, process)
      output_i = tf.expand_dims(output_i,0)
      if(v == 0):
        outputs_all_v = output_i
      else:
        outputs_all_v = tf.concat([outputs_all_v,output_i], axis = 0)

    View_feature_v = view_pooling_func(outputs_all_v, view_pooling)
    View_feature_v = tf.expand_dims(View_feature_v,0)

    if(i == 0):
      outputs_all = View_feature_v
    else:
      outputs_all = tf.concat([outputs_all,View_feature_v], axis = 0)

  outputs_all = tf.nn.dropout(outputs_all, dropout)

  logits  = tf.matmul(outputs_all, weights[-1]) + bias[-1]

  return logits  



### Train.py

In [ ]:
def get_loss( logits , gt_labels ):
    gt_labels = tf.stop_gradient(gt_labels) 
    return tf.nn.softmax_cross_entropy_with_logits(gt_labels,logits)
   

optimizer = tf.optimizers.Adam(learning_rate=0.001)

def batch_train(batch_inputs, batch_labels, A, dropout, process, view_pooling):
    with tf.GradientTape() as tape:
      logits = MV_model_infer(batch_inputs, A, dropout, process, view_pooling)
      # logits = model_infer(inputs_all, A, n_class, dropout, process)
      current_loss = get_loss(logits, batch_labels)

      trainable_variables = weights + bias

      lossL2 = tf.add_n([tf.nn.l2_loss(v) for v in trainable_variables])
      lossL2_decay = tf.reduce_mean(current_loss) + 0.001*lossL2

      grads = tape.gradient(lossL2_decay, trainable_variables)
      # grads = tape.gradient(current_loss, trainable_variables)
      
      # Update the weights
      optimizer.apply_gradients(zip(grads, trainable_variables))

    avg_loss = tf.reduce_mean(current_loss)

    return avg_loss


def train_all(Inputs_train, train_labels, A, n_class, batch_size, n_epoch, dropout, process, view_pooling):

  N_train = np.array(train_labels).size
  num_batches_per_epoch = int((N_train - 1) / batch_size) + 1
  ### Train
  for iter in range(n_epoch):
    epoch_loss = 0
    idx_all = np.random.permutation(N_train)
    start_idx = 0
    for i in range(num_batches_per_epoch):
      end_idx = start_idx + batch_size
      if(end_idx > N_train):
        end_idx = N_train 
      selected_idx = idx_all[start_idx:end_idx]

      train_label_select = train_labels[selected_idx]
      Inputs_select = Inputs_train[selected_idx,:,:,:]
      train_label_select_one_hot = tf.one_hot(train_label_select , depth=n_class)
      avg_loss = batch_train(Inputs_select, train_label_select_one_hot, A, dropout, process, view_pooling)
      epoch_loss += avg_loss

      start_idx += batch_size

    if(iter % 10 == 0 or iter == n_epoch - 1):  
      print('epoch = ',iter, 'loss = ', epoch_loss)

def train_all_and_validate(Inputs_train, train_labels, val_data, val_labels, test_data, test_labels, A, n_class, batch_size, n_epoch, dropout, process, view_pooling):

  N_train = np.array(train_labels).size
  num_batches_per_epoch = int((N_train - 1) / batch_size) + 1
  ### Train
  for iter in range(n_epoch):
    epoch_loss = 0
    idx_all = np.random.permutation(N_train)
    start_idx = 0
    for i in range(num_batches_per_epoch):
      end_idx = start_idx + batch_size
      if(end_idx > N_train):
        end_idx = N_train 
      selected_idx = idx_all[start_idx:end_idx]

      train_label_select = train_labels[selected_idx]
      Inputs_select = Inputs_train[selected_idx,:,:,:]
      train_label_select_one_hot = tf.one_hot(train_label_select , depth=n_class)
      avg_loss = batch_train(Inputs_select, train_label_select_one_hot, A, dropout, process, view_pooling)
      epoch_loss += avg_loss

      start_idx += batch_size

    accuracy_val, auc_val, sensitivity, specificity = validate(val_data, val_labels, A, dropout, process, view_pooling)  

    accuracy_test, auc_test, sensitivity, specificity = validate(test_data, test_labels, A, dropout, process, view_pooling)
  
    
    print('epoch = ',iter, 'loss = ', epoch_loss, 'val accuracy = ', accuracy_val, 'test accuracy = ', accuracy_test)    


def validate(Inputs_val, val_labels, A, dropout, process, view_pooling):
  logits = MV_model_infer(Inputs_val, A, dropout, process, view_pooling)
  predictions = tf.nn.softmax(logits)
  predicted_labels = np.argmax(predictions, axis = 1)
  predictions_auc = np.max(predictions, axis = 1)

  # print(predictions)
  # print(predicted_labels)
  # print(val_labels)

  accuracy, auc, sensitivity, specificity = cal_metrics(val_labels, predicted_labels, predicted_labels)


  return accuracy, auc, sensitivity, specificity




### Cal_metrics

In [ ]:
from sklearn.metrics import confusion_matrix
import sklearn

def cal_metrics(gt_label, predicted_label, predictions):
  if(np.array(predicted_label).size != np.array(gt_label).size):
    print('wrong label size')
    print('wrong label size')
    print('wrong label size')
    print('wrong label size')
    print('wrong label size')
    print('wrong label size')


  fpr, tpr, _ = sklearn.metrics.roc_curve(gt_label, predictions)
  auc = 100 * sklearn.metrics.auc(fpr, tpr)
  tn, fp, fn, tp = confusion_matrix(gt_label, predicted_label).ravel()
  sensitivity = tp/(tp+fn)
  specificity = tn/(tn+fp) 

  sum_correct = 0
  num_labels = np.array(predicted_label).size

  for i in range(num_labels):
    gt_i = gt_label[i]
    predicted_i = predicted_label[i]
    if(gt_i == predicted_i):
      sum_correct += 1

  accuracy = sum_correct/num_labels

  return accuracy, auc, sensitivity, specificity


def cal_intersection_union(array_1, array_2):
  array_1 = np.array(array_1)
  array_2 = np.array(array_2)

  intersect = np.intersect1d(array_1, array_2)

  num_intersect = intersect.size
  if(num_intersect>0):
    print('************************************')
    print('oho, there is intersection')
    print('oho, there is intersection')
    print('oho, there is intersection')
    print('oho, there is intersection')
    print('oho, there is intersection')
    print('oho, there is intersection')
    print('oho, there is intersection')
    print('************************************')

  union = np.union1d(array_1, array_2)
  num_union = union.size

  union_array = np.array(range(0, num_union))

  error = np.sum(union - union_array)

  if(error > 0):
    print('************************************')
    print('oho, wrong union')
    print('oho, wrong union')
    print('oho, wrong union')
    print('oho, wrong union')
    print('oho, wrong union')
    print('oho, wrong union')
    print('oho, wrong union')
    print('************************************')




### Train HOSVD Feature

In [ ]:
!pip install tensorly
import tensorly as tl
import time
from  scipy import *

def train_U_with_noise(data, R, train_idx):
  data_chosen = data[train_idx,:,:,:]
  X1 = tl.unfold(data_chosen, mode=1)
  B = np.matmul(X1,X1.transpose())
  U, S, V = np.linalg.svd(B, full_matrices=True)
  S = np.sqrt(S)
  sum_all_S = np.sum(S)
  len_S = len(S)
  sum_part_S = 0
  for i in range(len_S):
    sum_part_S = sum_part_S + S[i]
    if(sum_part_S>sum_all_S*R):
        break

  U = U[:,0:i]
  return U

def train_U_new(data, R):
  N,V,M,F = data.shape
  sum_X = 0 
  for i in range(N):
    for v in range(V):
      X_iv = data[i,v,:,:]
      sum_X = sum_X + np.matmul(X_iv, X_iv.transpose())

  U, S, V = np.linalg.svd(sum_X, full_matrices=True)   
  S = np.sqrt(S)
  sum_all_S = np.sum(S)
  len_S = len(S)
  sum_part_S = 0
  for i in range(len_S):
    sum_part_S = sum_part_S + S[i]
    if(sum_part_S>sum_all_S*R):
        break
  if(R == 1):
    U = U
  else:   
    U = U[:,0:i]

  return U   

def train_U(data_name, R):
  if(data_name == 'BP'):
    x = scipy.io.loadmat('BP.mat')
    X_normalize = x['X_normalize']

  elif(data_name == 'HIV'):
    x = scipy.io.loadmat('HIV.mat') 
    X_normalize = x['X'] 

  elif(data_name == 'PPMI'):
    x = scipy.io.loadmat('PPMI.mat') 
    X_normalize = x['X'] 

  N = X_normalize.size
  X_1 = X_normalize[0][0]
  M,F,V = X_1.shape
  # print('loading data:',data_name,'of size:',N,V,M,F)
  data = np.zeros([N,M,F,V])
  for i in range(N):
    X_i = X_normalize[i][0]
    for j in range(V):
      X_ij = X_i[:,:,j]
      data[i,:,:,j] = X_ij

  data_select = data
  X1 = tl.unfold(data_select, mode=1)
  B = np.matmul(X1,X1.transpose())
  U, S, V = np.linalg.svd(B, full_matrices=True)
  S = np.sqrt(S)
  sum_all_S = np.sum(S)
  len_S = len(S)
  sum_part_S = 0
  for i in range(len_S):
    sum_part_S = sum_part_S + S[i]
    if(sum_part_S>sum_all_S*R):
        break
  if(R == 1):
    U = U
  else:   
    U = U[:,0:i]
  return U


### Main.py

In [ ]:
import time
localtime = time.asctime( time.localtime(time.time()) )
print ("Local time :", localtime)
print('##$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$###############$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$*****************')


########## Initialization

method = 'gcn'
n_class = 2
data_name = 'BP'

data, _, _, _, train_index_all, test_index_all, label_all = load_data_my_new(data_name, 0)
data = np.array(data, dtype= 'float32')
label_all = label_all[:,0]
N,n_views,M,F = data.shape

index_set_all = np.array(range(N))
R = 1

node_features = train_U_new(data, R)
U = node_features
data = data_projection(data, U)
data = np.array(data,dtype='float32')

num_layers = 2

process = 'concat'
view_pooling = 'max'

dropout = 0.1

iter = 0
kfold = 10

accuracy_total_sum = 0
auc_total_sum = 0
sensitivity_total_sum = 0
specificity_total_sum = 0

accuracy_total_record = []
auc_total_record = []
sensitivity_total_record = []
specificity_total_record = []

n_epoch = 3
batch_size = 6

########################## Finish Initialization #####################

for train_fold_chosen in range(kfold):
  print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
  print('start training the model of fold:',train_fold_chosen)
  print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
  _, _, indexes_set, labels_set, _, _, _ = load_data_my_new(data_name, train_fold_chosen)
  iter = 0
  indexes = indexes_set[0]
  labels = labels_set[0]

  accuracy_record = []
  param_record = {}

  train_idx,val_idx,_ = indexes
  train_idx = train_index_all[train_fold_chosen]
  train_labels, val_labels = label_all[train_idx], label_all[val_idx]
  train_data,val_data = data[train_idx,:,:,:], data[val_idx,:,:,:]
  test_index = test_index_all[train_fold_chosen]
  test_data, test_labels = data[test_index,:,:,:], label_all[test_index]
  cal_intersection_union(train_idx, test_index)

  for Out_dim1 in range(30,50,20):
      if(num_layers == 3):
        Input_dim = F
        o_dim1 = Out_dim1
        o_dim2 = o_dim1
        o_dim3 = n_class
        Output_dim = [o_dim1,o_dim2,o_dim3]
      elif(num_layers == 2):
        Input_dim = F  
        o_dim1 = Out_dim1
        o_dim2 = n_class
        Output_dim = [o_dim1,o_dim2]
      shape_1 = [F, Out_dim1]
      shape_2 = [M*Out_dim1, 1800]
      shape_3 = [1800, n_class]
      shapes = [] 
      shapes.append(shape_1)
      shapes.append(shape_2)
      shapes.append(shape_3) 
      weights, bias = initialize_bias_weights(shapes)

      for knn in range(6,8,8):
        A = obtain_normalized_adj(node_features,knn)
        if(isspmatrix(A)):
          A = A.todense()
        A = np.array(A,dtype= 'float32')
        train_all_and_validate(train_data, train_labels, val_data, val_labels, test_data, test_labels, A, n_class, batch_size, 39, dropout, process, view_pooling)
        accuracy, auc, sensitivity, specificity = validate(test_data, test_labels, A, dropout, process, view_pooling)
        print('Out_dim1 = ', Out_dim1, 'knn = ',knn, 'validate accuracy = ', accuracy, 'batch_size =', batch_size, 'dropout = ', dropout, 'process = ', process, 'view_pooling = ', view_pooling)



  print('test fold = ', train_fold_chosen,' Out_dim1 = ', Out_dim1, 'knn = ',knn, 
        'accuracy = ', accuracy, 'auc = ', auc, 'sensitivity = ', sensitivity, 'specificity = ', specificity)

  del weights
  del bias


  accuracy_total_sum = accuracy_total_sum + accuracy
  auc_total_sum = auc_total_sum + auc
  sensitivity_total_sum = sensitivity_total_sum + sensitivity
  specificity_total_sum = specificity_total_sum + specificity

  accuracy_total_record.append(accuracy)
  auc_total_record.append(auc)
  sensitivity_total_record.append(sensitivity)
  specificity_total_record.append(specificity)

avg_accuracy, avg_auc =  accuracy_total_sum/kfold, auc_total_sum/kfold
avg_sensitivity, avg_specificity = sensitivity_total_sum/kfold, specificity_total_sum/kfold

print('%%%%%%%%%%%%%%%%% ')
print(' All Stats ')
print('%%%%%%%%%%%%%%%%% ')
print('avg_acc:', avg_accuracy, 'avg_auc:', avg_auc, 'avg_sensi:', avg_sensitivity, 'avg_speci:', avg_specificity)

print(np.array(accuracy_total_record))
print(np.array(auc_total_record))
print(np.array(sensitivity_total_record))
print(np.array(specificity_total_record))

### Main New Iter

In [ ]:
import time
localtime = time.asctime( time.localtime(time.time()) )
print ("Local time :", localtime)
print('##$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$###############$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$*****************')


########## Initialization

method = 'gcn'
n_class = 2
data_name = 'BP'

data, _, _, _, train_index_all, test_index_all, label_all = load_data_my_new(data_name, 0)
data = np.array(data, dtype= 'float32')
label_all = label_all[:,0]
N,n_views,M,F = data.shape

index_set_all = np.array(range(N))
R = 1

node_features = train_U_new(data, R)

U = node_features
data = data_projection(data, U)
data = np.array(data,dtype='float32')

N,n_views,M,F = data.shape


num_layers = 2

process = 'concat'
view_pooling = 'max'

dropout = 0.1

iter = 0
kfold = 10

accuracy_total_record = []
auc_total_record = []
sensitivity_total_record = []
specificity_total_record = []

n_epoch = 3
batch_size = 6

acc_iter_all = 0
auc_iter_all = 0
sensi_iter_all = 0
speci_iter_all = 0
########################## Finish Initialization #####################
for iter in range(1):
  accuracy_total_sum = 0
  auc_total_sum = 0
  sensitivity_total_sum = 0
  specificity_total_sum = 0
  for train_fold_chosen in range(kfold):
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('start training the model of fold:',train_fold_chosen)
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    _, _, indexes_set, labels_set, _, _, _ = load_data_my_new(data_name, train_fold_chosen)
    indexes = indexes_set[0]
    labels = labels_set[0]

    accuracy_record = []
    param_record = {}

    train_idx,val_idx,_ = indexes
    train_labels, val_labels,_ = labels
    train_data,val_data = data[train_idx,:,:,:], data[val_idx,:,:,:]
    test_index = test_index_all[train_fold_chosen]
    test_data, test_labels = data[test_index,:,:,:], label_all[test_index]

    for Out_dim1 in range(30,50,20):
        if(num_layers == 3):
          Input_dim = F
          o_dim1 = Out_dim1
          o_dim2 = o_dim1
          o_dim3 = n_class
          Output_dim = [o_dim1,o_dim2,o_dim3]
        elif(num_layers == 2):
          Input_dim = F  
          o_dim1 = Out_dim1
          o_dim2 = n_class
          Output_dim = [o_dim1,o_dim2]
        shape_1 = [F, Out_dim1]
        shape_2 = [M*Out_dim1, 1800]
        shape_3 = [1800, n_class]
        shapes = [] 
        shapes.append(shape_1)
        shapes.append(shape_2)
        shapes.append(shape_3) 


        for knn in range(6,8,2):
          A = obtain_normalized_adj(node_features,knn)

    print('##################################################################')
    print('start Retraining and testing the model with fold:',train_fold_chosen)
    print('Out_dim1 = ', Out_dim1, 'knn = ',knn, 'batch_size =', batch_size, 'dropout = ', dropout, 'process = ', process, 'view_pooling = ', view_pooling)
    print('##################################################################')


    train_index = train_index_all[train_fold_chosen]
    train_data, train_labels = data[train_index,:,:,:], label_all[train_index]
    print('train_data_shape = ', train_data.shape)
    cal_intersection_union(train_index, test_index)

    print('# of test: ', np.array(test_index).size)

    weights, bias = initialize_bias_weights(shapes)

    A = obtain_normalized_adj(node_features, knn)
    if(isspmatrix(A)):
      A = A.todense()
    A = np.array(A,dtype= 'float32')  

    train_all_and_validate(train_data, train_labels, val_data, val_labels, test_data, test_labels, A, n_class, batch_size, 39, dropout, process, view_pooling)

    accuracy, auc, sensitivity, specificity = validate(test_data, test_labels, A, dropout, process, view_pooling)

    print('test fold = ', train_fold_chosen,' Out_dim1 = ', Out_dim1, 'knn = ',knn, 
          'accuracy = ', accuracy, 'auc = ', auc, 'sensitivity = ', sensitivity, 'specificity = ', specificity)

    del weights
    del bias

    accuracy_total_sum = accuracy_total_sum + accuracy
    auc_total_sum = auc_total_sum + auc/100
    sensitivity_total_sum = sensitivity_total_sum + sensitivity
    specificity_total_sum = specificity_total_sum + specificity

  avg_accuracy, avg_auc =  accuracy_total_sum/kfold, auc_total_sum/kfold
  avg_sensitivity, avg_specificity = sensitivity_total_sum/kfold, specificity_total_sum/kfold

  print('iter = ', iter, 'avg_acc:', avg_accuracy, 'avg_auc:', avg_auc, 'avg_sensi:',avg_sensitivity, 'avg_speci:', avg_specificity)
  
  acc_iter_all += avg_accuracy
  auc_iter_all += avg_auc
  sensi_iter_all += avg_sensitivity
  speci_iter_all += avg_specificity

  accuracy_total_record.append(avg_accuracy)
  auc_total_record.append(avg_auc)
  sensitivity_total_record.append(avg_sensitivity)
  specificity_total_record.append(avg_specificity)

print('%%%%%%%%%%%%%%%%%')
print(' All Stats ')
print('%%%%%%%%%%%%%%%%%')
print('avg_acc:', acc_iter_all/(iter+1), 'avg_auc:', auc_iter_all/(iter+1), 'avg_sensi:', sensi_iter_all/(iter+1), 'avg_speci:', speci_iter_all/(iter+1))

print('%%%%%%%%%%%%%%%%%')
print(' All Stats ')
print('%%%%%%%%%%%%%%%%% ')
print('acc = ', accuracy_total_record)
print('auc = ', auc_total_record)
print('sensi = ', sensitivity_total_record)
print('speci = ', specificity_total_record)